In [1]:
import os
import glob
from PIL import Image
# from ultralytics import YOLO

In [2]:
# Example usage:
BKAI_train_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/train_images'
BKAI_labels_folder = '/kaggle/input/bkai-dataset2/vietnamese/labels'

BKAI_test_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/test_image'

BKAI_train_output_images_folder = "/kaggle/working/images/train"
BKAI_train_output_labels_folder = "/kaggle/working/labels/train"

BKAI_val_output_images_folder = "/kaggle/working/images/val"
BKAI_val_output_labels_folder = "/kaggle/working/labels/val"

In [3]:
import os
import shutil
import cv2

def ensure_directories_exist(directories):
    """
    Ensure that all directories in the given list exist.
    """
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

def convert_bbox_to_yolo(bbox, img_width, img_height):
    """
    Convert bounding box from absolute coordinates to YOLO format.
    bbox: [x1, y1, x2, y2, x3, y3, x4, y4]
    """
    x_coords = bbox[0::2]  # [x1, x2, x3, x4]
    y_coords = bbox[1::2]  # [y1, y2, y3, y4]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    
    x_center = (x_min + x_max) / 2.0 / img_width
    y_center = (y_min + y_max) / 2.0 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return x_center, y_center, width, height

def process_labels_and_images(
    images_folder, labels_folder, output_images_folder, output_labels_folder
):
    """
    Process images and their corresponding label files, converting bounding boxes to YOLO format.
    """
    ensure_directories_exist([output_images_folder, output_labels_folder])
    
    for image_name in os.listdir(images_folder):
        if not image_name.endswith('.jpg'):
            continue

        # Map image to label
        image_id = str(int(image_name.split('.')[0].replace("im", "")))  # e.g., im0001 -> 1
        label_file = os.path.join(labels_folder, f"gt_{image_id}.txt")
        if not os.path.exists(label_file):
            continue

        # Read image dimensions
        image_path = os.path.join(images_folder, image_name)
        img = cv2.imread(image_path)
        img_height, img_width = img.shape[:2]

        # Process label file
        yolo_labels = []
        with open(label_file, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split(',')
                bbox = list(map(int, parts[:8]))  # First 8 values are bbox
                label = parts[8]  # Last value is the text/label

                # Convert bbox to YOLO format
                x_center, y_center, width, height = convert_bbox_to_yolo(bbox, img_width, img_height)
                yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")

        # Save image to output folder
        shutil.copy(image_path, os.path.join(output_images_folder, image_name))

        # Save YOLO labels to output folder
        output_label_file = os.path.join(output_labels_folder, f"{image_id}.txt")
        with open(output_label_file, 'w') as f:
            f.write("\n".join(yolo_labels))






In [4]:
# Example usage:
BKAI_train_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/train_images'
BKAI_labels_folder = '/kaggle/input/bkai-dataset2/vietnamese/labels'

BKAI_val_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/test_image'

BKAI_test_images_folder = '/kaggle/input/bkai-dataset2/vietnamese/unseen_test_images'

BKAI_train_output_images_folder = "/kaggle/working/train/images"
BKAI_train_output_labels_folder = "/kaggle/working/train/labels"

BKAI_val_output_images_folder = "/kaggle/working/val/images"
BKAI_val_output_labels_folder = "/kaggle/working/val/labels"

BKAI_test_output_images_folder = "/kaggle/working/test/images"
BKAI_test_output_labels_folder = "/kaggle/working/test/labels"

In [5]:
process_labels_and_images(
    BKAI_train_images_folder, 
    BKAI_labels_folder, 
    BKAI_train_output_images_folder, 
    BKAI_train_output_labels_folder
)

In [6]:
process_labels_and_images(
    BKAI_val_images_folder, 
    BKAI_labels_folder, 
    BKAI_val_output_images_folder, 
    BKAI_val_output_labels_folder
)

In [7]:
process_labels_and_images(
    BKAI_test_images_folder, 
    BKAI_labels_folder, 
    BKAI_test_output_images_folder, 
    BKAI_test_output_labels_folder
)